# Прогноз стоимости квартиры моих знакомых из г. Петропавловска-Камчатского

загружаем необходимые библиотеки

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Исходные данные:
### из базы Циан получена информация о находящихся в продаже квартирах (вторичный рынок) в Петропавловске-Камчатском и в близлежащих населённых пунктах - Дальнем, Нагорном, и т.п.
Объявления сгруппированы по территориальному признаку, оказывающими влияние на цену картиры и отнесены к одному из 15 районов: центр, ул. Чубарова с окрестностями, населённые пункты Петропавловск-Камчтского городского округа, Горизонт, КП, Моховая (включая 10-11 км), ул. Океанская, населёные пункты Елизовского района, ул. Рябикова с окрестностями, Северо-Восток, Серголазка (от КП до Космонавтов и Фестивальной), СРВ-ЖБФ, ул. Солнечная с окрестностями, мкр Завойко, ул. Зеркальная, ул. Владивостокская с окрестностями (от ост. УВД до 5 километра примерно).

Также к значимым признакам отнесено количество комнат, общая площадьквартиры, этаж, на котром находится квартира и общее количество этажей в доме, тип сделки (свободная/альтернативная), возможность ипотеки, наличие ремонта (полный, косметический), наличие и количество балконов и лоджий, количество и тип (совмещённый/раздельный) санузлов, наличие стационального телефона.

В базе Циан отсутствует информация о годе постройки домов и материале стен, к сожалению.

In [2]:
df = pd.read_csv(r'https://docs.google.com/spreadsheets/d/e/2PACX-1vRlV_-keB5GeDNU8M_ijQeMGokUqLnyiAm7Yh0XygD7ddn_6Kx5d0wn16ooLG2UBNx3Ui3lzUKU1tGn/pub?gid=1057764682&single=true&output=csv', sep = ',').drop(columns = ['id'], axis = 1)


In [3]:
df

,cen,chu,dal,gor,kpl,moh,oke,pri,ryb,s_v,...,free,alt,mort,ren_full,ren_cosm,balcony,loggia,wc,wc_sep,tel
0,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
1,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,1
2,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,1
3,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
4,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,0,1,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,1,0,1
426,0,1,0,0,0,0,0,0,0,0,...,1,0,1,1,0,1,0,1,0,0
427,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,1,0,1,0,0
428,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,1,1,0,1,0


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 30 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   cen       430 non-null    int64  
 1   chu       430 non-null    int64  
 2   dal       430 non-null    int64  
 3   gor       430 non-null    int64  
 4   kpl       430 non-null    int64  
 5   moh       430 non-null    int64  
 6   oke       430 non-null    int64  
 7   pri       430 non-null    int64  
 8   ryb       430 non-null    int64  
 9   s_v       430 non-null    int64  
 10  ser       430 non-null    int64  
 11  srv       430 non-null    int64  
 12  sun       430 non-null    int64  
 13  zav       430 non-null    int64  
 14  zer       430 non-null    int64  
 15  rooms     430 non-null    int64  
 16  squ       430 non-null    float64
 17  fl_a      430 non-null    int64  
 18  fl_h      430 non-null    int64  
 19  price     430 non-null    float64
 20  free      430 non-null    int64 

In [5]:
df.describe()

,cen,chu,dal,gor,kpl,moh,oke,pri,ryb,s_v,...,free,alt,mort,ren_full,ren_cosm,balcony,loggia,wc,wc_sep,tel
count,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,...,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000
mean,0.127907,0.053488,0.027907,0.120930,0.037209,0.072093,0.055814,0.030233,0.030233,0.097674,...,0.981395,0.018605,0.383721,0.141860,0.555814,0.537209,0.062791,0.604651,0.481395,0.355814
std,0.334375,0.225267,0.164898,0.326426,0.189495,0.258943,0.229829,0.171426,0.171426,0.297220,...,0.135281,0.135281,0.486858,0.349313,0.497454,0.543889,0.302687,0.512753,0.609465,0.479317
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,4.000000,2.000000,4.000000,1.000000


## Формируем признаки и целевую переменную

In [6]:
X = df[['cen', 'chu', 'dal', 'gor', 'kpl', 'moh', 'oke', 'pri', 'ryb', 's_v', 'ser', 'srv', 'sun', 'zav', 'zer',	'rooms',	'squ',	'fl_a',	'fl_h', 'free',	'alt',	'mort',	'ren_full',	'ren_cosm',	'balcony',	'loggia',	'wc',	'wc_sep',	'tel']]
y = df['price']

## Разделяем данные на выборку для обучения/проверки

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Создаем и обучаем модель линейной регресии

In [8]:
lr = LinearRegression()

In [9]:
lr.fit(X_train, y_train)

LinearRegression()

In [10]:
lr.coef_

array([ 1401251.66121935,  1091849.86943074, -1732763.12734504,
         586942.60545683,   532058.6919797 ,   772924.2846788 ,
        -443706.34055222,    16745.8307728 , -1008526.23980808,
        1653185.76378755,   417053.19050623,  -918181.81579168,
        -430033.24384308, -2397114.70937194,   458313.57888001,
          23197.21931477,   101454.06224448,    62222.33699735,
         182242.31477091,   243629.39068628,  -243629.39068628,
         139385.25256559,  1433811.13170885,   594660.63986547,
         193660.70612229,   512907.77240975,   514618.07218251,
         415615.86920591,  -215890.63668241])

## Получаем предсказание и оцениваем качество

In [11]:
lr.predict(X_test)

array([ 6539060.247138  ,  5235991.89051982,  3961225.5978828 ,
        6292416.16482496,  7677693.28819862,  7490270.55438017,
        5988003.20758925,  6535507.71832247,  6415085.17139758,
        6948036.21542237,  2911481.46059982,  4518227.81124508,
        3926480.88611995,  3529927.58389895,  6710066.96332316,
        4101100.03052929,  5833702.27729722,  8401379.0197112 ,
        7330002.10556055,  6170708.91001828,  7088929.76265663,
        5145684.9084689 ,  4145191.96280436,  4586005.644131  ,
        3119337.27189854,  5506744.74587128,  6271416.05595229,
        5437844.76315143,  6566131.81273833,  5682373.03891925,
        5503141.90328334,  6954444.64283548,  6052897.13753247,
        5379494.28174703,  3915742.80799463,  6847713.28493434,
        7322955.82652776,  5156976.24258456,  8130651.34182957,
        7819245.1377251 ,  4932872.43417554,  9352860.59528029,
        5869589.90515103,  7705007.13859598,  4067419.82683925,
        5059413.90634121,  5246101.16145

In [12]:
# Оценим качество, при помощи метода mean_squared_error для тестовой выборки
mean_squared_error(y_test, lr.predict(X_test))

1269994801229.2554

In [13]:
mean_squared_error(y_train, lr.predict(X_train))

1691731039876.366

## Делаем оценку для конкретной квартиры:

In [14]:
data = [{
    "cen": 0,
    "chu": 0,
    "dal": 0,
    "gor": 0,
    "kpl": 0,
    "moh": 0,
    "oke": 0,
    "pri": 0,
    "ryb": 0,
    "s_v": 1,
    "ser": 0,
    "srv": 0,
    "sun": 0,
    "zav": 0,
    "zer": 0,
    "rooms": 4,
    "squ": 92,
    "fl_a": 3,
    "fl_h": 5,
    "free": 1,
    "alt": 0,
    "mort": 1,
    "ren_full": 1,
    "ren_cosm": 0,
    "balcony": 2,
    "loggia": 0,
    "wc": 0,
    "wc_sep": 1,
    "tel": 1
}]

In [15]:
df_person = pd.DataFrame(data)
df_person.head()

,cen,chu,dal,gor,kpl,moh,oke,pri,ryb,s_v,...,free,alt,mort,ren_full,ren_cosm,balcony,loggia,wc,wc_sep,tel
0,0,0,0,0,0,0,0,0,0,1,...,1,0,1,1,0,2,0,0,1,1


In [16]:
lr.predict(df_person)

array([12404914.54926067])